# 6月研究発表

【余談】<br>
前回，スライドも別に作成して時間取ったので今回はRISEを使ってjupyterからスライド生成してみました．

### アジェンダ
- 前回の復習
- $P$を$P_j (j=1,2,\ldots , 9)$にしてみる
- 9イニングにおける得点分布

### 前回のコードを読み込み

In [2]:
# ライブラリの読み込み
import numpy as np
import pandas as pd
from tqdm import tqdm
from functools import lru_cache
p_s, p_d, p_t, p_h, p_w = 0.133, 0.032, 0.001, 0.065, 0.114
p_out = 1-(p_s+p_d+p_t+p_h+p_w)
A = np.array([[p_h, p_s+p_w, p_d, p_t, 0, 0, 0, 0],
              [p_h, 0, 0, p_t, p_s+p_w, 0, p_d, 0],
              [p_h, p_s, p_d, p_t, p_w, 0, 0, 0],
              [p_h, p_s, p_d, p_t, 0, p_w, 0, 0],
              [p_h, 0, 0, p_t, p_s, 0, p_d, p_w],
              [p_h, 0, 0, p_t, p_s, 0, p_d, p_w],
              [p_h, p_s, p_d, p_t, 0, 0, 0, p_w],
              [p_h, 0, 0, p_t, p_s, 0, p_d, p_w]])
F = np.array([np.repeat(p_out, 8)]).T
B = F * np.eye(8)
# 遷移確率行列
P = np.block([[A, B, np.zeros((8, 8)), np.zeros((8, 1))],
             [np.zeros((8, 8)), A, B, np.zeros((8, 1))],
             [np.zeros((8, 8)), np.zeros((8, 8)), A, F],
             [np.zeros((1, 8)), np.zeros((1, 8)), np.zeros((1, 8)), 1]])
# 初期値
u_0 = np.insert(np.repeat(0, 24), 0, 1)


def oneinning_sim(n):
    if n == 0:
        return u_0
    else:
        return np.dot(oneinning_sim(n-1), P)
A_0 = np.array([[0, p_s+p_w, p_d, p_t, 0, 0, 0, 0],
                [0, 0, 0, 0, p_s+p_w, 0, p_d, 0],
                [0, 0, 0, 0, p_w, 0, 0, 0],
                [0, 0, 0, 0, 0, p_w, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, p_w],
                [0, 0, 0, 0, 0, 0, 0, p_w],
                [0, 0, 0, 0, 0, 0, 0, p_w],
                [0, 0, 0, 0, 0, 0, 0, 0]])

A_1 = np.array([[p_h, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, p_t, 0, 0, 0, 0],
                [0, p_s, p_d, p_t, 0, 0, 0, 0],
                [0, p_s, p_d, p_t, 0, 0, 0, 0],
                [0, 0, 0, 0, p_s, 0, p_d, 0],
                [0, 0, 0, 0, p_s, 0, p_d, 0],
                [0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, p_w]])

A_2 = np.array([[0, 0, 0, 0, 0, 0, 0, 0],
                [p_h, 0, 0, 0, 0, 0, 0, 0],
                [p_h, 0, 0, 0, 0, 0, 0, 0],
                [p_h, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, p_t, 0, 0, 0, 0],
                [0, 0, 0, p_t, 0, 0, 0, 0],
                [0, p_s, p_d, p_t, 0, 0, 0, 0],
                [0, 0, 0, 0, p_s, 0, p_d, 0]])

A_3 = np.array([[0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0],
                [p_h, 0, 0, 0, 0, 0, 0, 0],
                [p_h, 0, 0, 0, 0, 0, 0, 0],
                [p_h, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, p_t, 0, 0, 0, 0]])

A_4 = np.array([[0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0],
                [p_h, 0, 0, 0, 0, 0, 0, 0]])

P_0 = np.block([[A_0, B, np.zeros((8, 8)), np.zeros((8, 1))],
                [np.zeros((8, 8)), A_0, B, np.zeros((8, 1))],
                [np.zeros((8, 8)), np.zeros((8, 8)), A_0, F],
                [np.zeros((1, 8)), np.zeros((1, 8)), np.zeros((1, 8)), 1]])

P_1 = np.block([[A_1, np.zeros((8, 8)), np.zeros((8, 8)), np.zeros((8, 1))],
                [np.zeros((8, 8)), A_1, np.zeros((8, 8)), np.zeros((8, 1))],
                [np.zeros((8, 8)), np.zeros((8, 8)), A_1, np.zeros((8, 1))],
                [np.zeros((1, 8)), np.zeros((1, 8)), np.zeros((1, 8)), 0]])

P_2 = np.block([[A_2, np.zeros((8, 8)), np.zeros((8, 8)), np.zeros((8, 1))],
                [np.zeros((8, 8)), A_2, np.zeros((8, 8)), np.zeros((8, 1))],
                [np.zeros((8, 8)), np.zeros((8, 8)), A_2, np.zeros((8, 1))],
                [np.zeros((1, 8)), np.zeros((1, 8)), np.zeros((1, 8)), 0]])

P_3 = np.block([[A_3, np.zeros((8, 8)), np.zeros((8, 8)), np.zeros((8, 1))],
                [np.zeros((8, 8)), A_3, np.zeros((8, 8)), np.zeros((8, 1))],
                [np.zeros((8, 8)), np.zeros((8, 8)), A_3, np.zeros((8, 1))],
                [np.zeros((1, 8)), np.zeros((1, 8)), np.zeros((1, 8)), 0]])

P_4 = np.block([[A_4, np.zeros((8, 8)), np.zeros((8, 8)), np.zeros((8, 1))],
                [np.zeros((8, 8)), A_4, np.zeros((8, 8)), np.zeros((8, 1))],
                [np.zeros((8, 8)), np.zeros((8, 8)), A_4, np.zeros((8, 1))],
                [np.zeros((1, 8)), np.zeros((1, 8)), np.zeros((1, 8)), 0]])
# 初期状態
R_max = 20

U_0 = np.zeros((R_max+1, 25))
U_0[0, 0] = 1

# 再帰関数
@lru_cache(maxsize=1000)
def inning_score_sim(n, r):
    if n == 0:
        return U_0[r]
    elif r == 0:
        return np.dot(inning_score_sim(n-1, r), P_0)
    elif r == 1:
        return np.dot(inning_score_sim(n-1, r), P_0) + np.dot(inning_score_sim(n-1, r-1), P_1)
    elif r == 2:
        return np.dot(inning_score_sim(n-1, r), P_0) + np.dot(inning_score_sim(n-1, r-1), P_1) + np.dot(inning_score_sim(n-1, r-2), P_2)
    elif r == 3:
        return np.dot(inning_score_sim(n-1, r), P_0) + np.dot(inning_score_sim(n-1, r-1), P_1) + np.dot(inning_score_sim(n-1, r-2), P_2) + np.dot(inning_score_sim(n-1, r-3), P_3)
    elif r >= 4:
        return np.dot(inning_score_sim(n-1, r), P_0) + np.dot(inning_score_sim(n-1, r-1), P_1) + np.dot(inning_score_sim(n-1, r-2), P_2) + np.dot(inning_score_sim(n-1, r-3), P_3) + np.dot(inning_score_sim(n-1, r-4), P_4)


# nを∞に発散
n = 3
U_n = []
for r in range(R_max+1):
    U_n.append(inning_score_sim(n, r))
df_U = pd.DataFrame(U_n, columns=list(range(1,26)))
df_U

,1,2,3,4,5,6,7,8,9,10,...,16,17,18,19,20,21,22,23,24,25
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008285,0.000000,0.000000,...,0.0,0.00000,0.317908,0.041186,0.001287,0.0,0.0,0.0,0.0,0.281011
1,0.000000,0.000000,0.000000,0.000000,0.014022,0.000039,0.002727,0.000000,0.000000,0.040172,...,0.0,0.08366,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
2,0.000000,0.007053,0.001082,0.000119,0.000000,0.000000,0.000000,0.000000,0.044065,0.000000,...,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
3,0.007737,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000


## $P$を動かす

$P$は本来，選手一人一人が固有で持っている遷移確率行列．打順$j$を考慮して１イニングのシミュレーションを行う．

In [3]:
# データ読み込み
df = pd.read_csv('player_data/batting.csv')
giants_2019 = df[(df['Team'] == 'Yomiuri Giants') & (df['Year'] == 2019)].reset_index(drop = True)
giants_2019.sort_values('G', ascending=False).head(5)

,Rk,Name,Age,G,PA,AB,R,H,2B,3B,...,GDP,HBP,SH,SF,IBB,League,Team,Year,PlayerID,Bats
25,26,Hayato Sakamoto,30.0,143,639,555,103,173,26,0,...,9.0,2,3,2.0,6.0,Japan Central League,Yomiuri Giants,2019,sakamo001hay,right
23,24,Kazuma Okamoto,23.0,143,628,555,84,147,29,0,...,15.0,6,2,3.0,3.0,Japan Central League,Yomiuri Giants,2019,okamot002kaz,right
17,18,Yoshihiro Maru*,30.0,143,631,535,82,156,26,1,...,15.0,3,0,7.0,3.0,Japan Central League,Yomiuri Giants,2019,maru--001yos,left
10,11,Yoshiyuki Kamei*,36.0,131,503,450,67,128,27,2,...,6.0,0,3,5.0,3.0,Japan Central League,Yomiuri Giants,2019,kamei-001yos,left
45,46,Daikan Yoh,32.0,110,231,208,28,57,8,1,...,9.0,1,0,0.0,0.0,Japan Central League,Yomiuri Giants,2019,yang--001chu,right


In [4]:
# 特徴量エンジニアリング
# 遷移確率行列を定義するのに必要な変数だけを残す

mask = ['Name', 'H', '2B', '3B', 'HR', 'BB', 'IBB', 'HBP', 'PA']
giants_2019 = giants_2019[mask]

# 単打の列を追加，四死球をまとめる
single = giants_2019['H'] - (giants_2019['2B'] + giants_2019['3B'] + giants_2019['HR'])
giants_2019.insert(2, '1B', single)
giants_2019['BB_HBP'] = giants_2019['BB'] + giants_2019['IBB'] + giants_2019['HBP']

# 不要になった変数を削除
giants_2019.drop(['H', 'BB', 'IBB', 'HBP'],axis=1,  inplace=True)

giants_2019.head(10)

,Name,1B,2B,3B,HR,PA,BB_HBP
0,Shinnosuke Abe*,32,8,0,7,192,34.0
1,Ryan Cook,0,0,0,0,1,0.0
2,Cristopher Crisostomo*,3,0,0,0,42,1.0
3,Yuri Furukawa,0,0,0,0,1,0.0
4,Alex Guerrero,29,16,2,21,333,46.0
5,Seishu Hatake*,1,0,0,0,4,1.0
6,Nobutaka Imamura*,4,0,0,0,26,1.0
7,Shingo Ishikawa,12,2,0,4,76,7.0
8,Taylor Jungmann,3,0,0,0,14,0.0
9,Yohei Kagiya,0,0,0,0,1,0.0


In [5]:
## 遷移確率を持つデータフレームを作成
def rate_convert(e):
    rate = giants_2019[e] / giants_2019["PA"]
    return rate

single_rate = rate_convert("1B")
double_rate = rate_convert("2B")
triple_rate = rate_convert("3B")
HR_rate = rate_convert("HR")
BB_HBP_rate = rate_convert('BB_HBP')
out_rate = 1 - (single_rate + double_rate + triple_rate +
                HR_rate + BB_HBP_rate)

team_rate = pd.DataFrame({"選手名": giants_2019['Name'], "単打": single_rate, "二塁打": double_rate,
                          "三塁打": triple_rate, "本塁打": HR_rate, "四死球": BB_HBP_rate, "アウト": out_rate})
# NPBの平均投手の推移確率を追加
pitcher = pd.DataFrame([["pitcher", 0.076, 0.014, 0.001, 0.005, 0.0323,
                      1-sum([0.076, 0.014, 0.001, 0.005, 0.0323])]], columns=team_rate.columns.values)
team_rate = pd.concat([team_rate, pitcher])

team_rate.sort_values('単打', ascending=False)


,選手名,単打,二塁打,三塁打,本塁打,四死球,アウト
47,Naoki Yoshikawa*,0.326087,0.021739,0.000000,0.000000,0.065217,0.586957
39,Shingo Usami*,0.250000,0.000000,0.000000,0.000000,0.000000,0.750000
5,Seishu Hatake*,0.250000,0.000000,0.000000,0.000000,0.250000,0.500000
8,Taylor Jungmann,0.214286,0.000000,0.000000,0.000000,0.000000,0.785714
45,Daikan Yoh,0.190476,0.034632,0.004329,0.017316,0.099567,0.653680
14,Seiji Kobayashi,0.186441,0.025424,0.000000,0.008475,0.046610,0.733051
28,Shinnosuke Shigenobu*,0.178161,0.040230,0.011494,0.011494,0.068966,0.689655
10,Yoshiyuki Kamei*,0.170974,0.053678,0.003976,0.025845,0.095427,0.650099
25,Hayato Sakamoto,0.167449,0.040689,0.000000,0.062598,0.133020,0.596244
24,Takumi Oshiro*,0.167173,0.048632,0.003040,0.018237,0.088146,0.674772


In [6]:
# 2019年の巨人打順にそって，遷移確率行列を作成する
team_order = ['Yoshiyuki Kamei*','Hayato Sakamoto','Yoshihiro Maru*','Kazuma Okamoto', 'Takumi Oshiro*','Akihiro Wakabayashi#', 'Shinnosuke Shigenobu*', 'Seiji Kobayashi','pitcher' ]

team_order_rate = pd.DataFrame(
    index=[], columns=team_rate.columns.values)

for order in team_order:
    #print(team_rate[team_rate['選手名'] == order])
    team_order_rate = pd.concat([team_order_rate,team_rate[team_rate['選手名'] == order]])

# 打順を付与
team_order_rate.index = np.arange(1, len(team_order_rate) + 1)
team_order_rate

,選手名,単打,二塁打,三塁打,本塁打,四死球,アウト
1,Yoshiyuki Kamei*,0.170974,0.053678,0.003976,0.025845,0.095427,0.650099
2,Hayato Sakamoto,0.167449,0.040689,0.0,0.062598,0.13302,0.596244
3,Yoshihiro Maru*,0.161648,0.041204,0.001585,0.042789,0.1458,0.606973
4,Kazuma Okamoto,0.138535,0.046178,0.0,0.049363,0.113057,0.652866
5,Takumi Oshiro*,0.167173,0.048632,0.00304,0.018237,0.088146,0.674772
6,Akihiro Wakabayashi#,0.150183,0.032967,0.003663,0.018315,0.128205,0.666667
7,Shinnosuke Shigenobu*,0.178161,0.04023,0.011494,0.011494,0.068966,0.689655
8,Seiji Kobayashi,0.186441,0.025424,0.0,0.008475,0.04661,0.733051
9,pitcher,0.076,0.014,0.001,0.005,0.0323,0.8717


In [7]:
## 選手固有の遷移確率行列
P_j = []

for idx, row in team_order_rate.iterrows():
    p_h, p_d, p_t, p_h, p_w, p_out = row['単打'], row['二塁打'], row['三塁打'], row['本塁打'], row['四死球'], row['アウト']
    A = np.array([[p_h, p_s+p_w, p_d, p_t, 0, 0, 0, 0],
              [p_h, 0, 0, p_t, p_s+p_w, 0, p_d, 0],
              [p_h, p_s, p_d, p_t, p_w, 0, 0, 0],
              [p_h, p_s, p_d, p_t, 0, p_w, 0, 0],
              [p_h, 0, 0, p_t, p_s, 0, p_d, p_w],
              [p_h, 0, 0, p_t, p_s, 0, p_d, p_w],
              [p_h, p_s, p_d, p_t, 0, 0, 0, p_w],
              [p_h, 0, 0, p_t, p_s, 0, p_d, p_w]])
    F = np.array([np.repeat(p_out, 8)]).T
    B = F * np.eye(8)
    # 遷移確率行列
    P = np.block([[A, B, np.zeros((8, 8)), np.zeros((8, 1))],
             [np.zeros((8, 8)), A, B, np.zeros((8, 1))],
             [np.zeros((8, 8)), np.zeros((8, 8)), A, F],
             [np.zeros((1, 8)), np.zeros((1, 8)), np.zeros((1, 8)), 1]])
    P_j.append(P)
print(P_j[0])

[[0.02584493 0.22842744 0.05367793 0.00397614 0.         0.
  0.         0.         0.6500994  0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]
 [0.02584493 0.         0.         0.00397614 0.22842744 0.
  0.05367793 0.         0.         0.6500994  0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]
 [0.02584493 0.133      0.05367793 0.00397614 0.09542744 0.
  0.         0.         0.         0.         0.6500994  0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]
 [0.02584493 0.133      0.05367793 0.00397614 0.         0.09542744
  0.         0.         0.         0.         0.         0.6500994
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.      

In [8]:
# 初期値
u_0 = np.insert(np.repeat(0, 24), 0, 1)

def oneinning_sim(n, i):
    if n == 0:
        return u_0
    else:
        if i < 8:
            i+=1
        else:
            i=0
        return np.dot(oneinning_sim(n-1, i), P_j[i])
        

# n人目の打者のプレーが終わったとき状態ｓに至っている確率を要素に持つベクトル
print(oneinning_sim(10, -1))

[3.57758412e-07 7.38801790e-07 2.07809847e-07 5.50397557e-08
 1.52793547e-06 0.00000000e+00 5.35226855e-07 8.95269189e-07
 9.55378800e-06 1.74769632e-05 4.84255304e-06 1.23514611e-06
 3.76726748e-05 2.33731233e-08 1.28233893e-05 2.22677073e-05
 1.12129775e-04 1.84153769e-04 5.00418231e-05 1.21641452e-05
 4.09374985e-04 4.47845641e-07 1.35612845e-04 2.47335080e-04
 9.27642912e-01]


In [9]:
## 遷移確率行列の分解
P_j_k = []

for idx, row in team_order_rate.iterrows():
    p_h, p_d, p_t, p_h, p_w, p_out = row['単打'], row['二塁打'], row['三塁打'], row['本塁打'], row['四死球'], row['アウト']
    P_k = []
    A_0 = np.array([[0, p_s+p_w, p_d, p_t, 0, 0, 0, 0],
                    [0, 0, 0, 0, p_s+p_w, 0, p_d, 0],
                    [0, 0, 0, 0, p_w, 0, 0, 0],
                    [0, 0, 0, 0, 0, p_w, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, p_w],
                    [0, 0, 0, 0, 0, 0, 0, p_w],
                    [0, 0, 0, 0, 0, 0, 0, p_w],
                    [0, 0, 0, 0, 0, 0, 0, 0]])

    A_1 = np.array([[p_h, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, p_t, 0, 0, 0, 0],
                    [0, p_s, p_d, p_t, 0, 0, 0, 0],
                    [0, p_s, p_d, p_t, 0, 0, 0, 0],
                    [0, 0, 0, 0, p_s, 0, p_d, 0],
                    [0, 0, 0, 0, p_s, 0, p_d, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, p_w]])

    A_2 = np.array([[0, 0, 0, 0, 0, 0, 0, 0],
                    [p_h, 0, 0, 0, 0, 0, 0, 0],
                    [p_h, 0, 0, 0, 0, 0, 0, 0],
                    [p_h, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, p_t, 0, 0, 0, 0],
                    [0, 0, 0, p_t, 0, 0, 0, 0],
                    [0, p_s, p_d, p_t, 0, 0, 0, 0],
                    [0, 0, 0, 0, p_s, 0, p_d, 0]])

    A_3 = np.array([[0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0],
                    [p_h, 0, 0, 0, 0, 0, 0, 0],
                    [p_h, 0, 0, 0, 0, 0, 0, 0],
                    [p_h, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, p_t, 0, 0, 0, 0]])

    A_4 = np.array([[0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0],
                    [p_h, 0, 0, 0, 0, 0, 0, 0]])

    P_0 = np.block([[A_0, B, np.zeros((8, 8)), np.zeros((8, 1))],
                    [np.zeros((8, 8)), A_0, B, np.zeros((8, 1))],
                    [np.zeros((8, 8)), np.zeros((8, 8)), A_0, F],
                    [np.zeros((1, 8)), np.zeros((1, 8)), np.zeros((1, 8)), 1]])
    P_k.append(P_0)
    P_1 = np.block([[A_1, np.zeros((8, 8)), np.zeros((8, 8)), np.zeros((8, 1))],
                    [np.zeros((8, 8)), A_1, np.zeros((8, 8)), np.zeros((8, 1))],
                    [np.zeros((8, 8)), np.zeros((8, 8)), A_1, np.zeros((8, 1))],
                    [np.zeros((1, 8)), np.zeros((1, 8)), np.zeros((1, 8)), 0]])
    P_k.append(P_1)
    P_2 = np.block([[A_2, np.zeros((8, 8)), np.zeros((8, 8)), np.zeros((8, 1))],
                    [np.zeros((8, 8)), A_2, np.zeros((8, 8)), np.zeros((8, 1))],
                    [np.zeros((8, 8)), np.zeros((8, 8)), A_2, np.zeros((8, 1))],
                    [np.zeros((1, 8)), np.zeros((1, 8)), np.zeros((1, 8)), 0]])
    P_k.append(P_2)
    P_3 = np.block([[A_3, np.zeros((8, 8)), np.zeros((8, 8)), np.zeros((8, 1))],
                    [np.zeros((8, 8)), A_3, np.zeros((8, 8)), np.zeros((8, 1))],
                    [np.zeros((8, 8)), np.zeros((8, 8)), A_3, np.zeros((8, 1))],
                    [np.zeros((1, 8)), np.zeros((1, 8)), np.zeros((1, 8)), 0]])
    P_k.append(P_3)
    P_4 = np.block([[A_4, np.zeros((8, 8)), np.zeros((8, 8)), np.zeros((8, 1))],
                    [np.zeros((8, 8)), A_4, np.zeros((8, 8)), np.zeros((8, 1))],
                    [np.zeros((8, 8)), np.zeros((8, 8)), A_4, np.zeros((8, 1))],
                    [np.zeros((1, 8)), np.zeros((1, 8)), np.zeros((1, 8)), 0]])
    P_k.append(P_4)
    P_j_k.append(P_k)

In [17]:
# 初期状態
R_max = 20

U_0 = np.zeros((R_max+1, 25))
U_0[0, 0] = 1

# 再帰関数
@lru_cache(maxsize=1000)
def inning_score_sim(n, r , i):
    if n == 0:
        return U_0[r]
    elif r == 0:
        return np.dot(inning_score_sim(n-1, r, i), P_j_k[i][0])
    elif r == 1:
        return np.dot(inning_score_sim(n-1, r, i), P_j_k[i][0]) + np.dot(inning_score_sim(n-1, r-1, i), P_j_k[i][1])
    elif r == 2:
        return np.dot(inning_score_sim(n-1, r, i), P_j_k[i][0]) + np.dot(inning_score_sim(n-1, r-1, i), P_j_k[i][1]) + np.dot(inning_score_sim(n-1, r-2, i), P_j_k[i][2])
    elif r == 3:
        return np.dot(inning_score_sim(n-1, r, i), P_j_k[i][0]) + np.dot(inning_score_sim(n-1, r-1, i), P_j_k[i][1]) + np.dot(inning_score_sim(n-1, r-2, i), P_j_k[i][2]) + np.dot(inning_score_sim(n-1, r-3, i), P_j_k[i][3])
    elif r >= 4:
        return np.dot(inning_score_sim(n-1, r, i), P_j_k[i][0]) + np.dot(inning_score_sim(n-1, r-1, i), P_j_k[i][1]) + np.dot(inning_score_sim(n-1, r-2, i), P_j_k[i][2]) + np.dot(inning_score_sim(n-1, r-3, i), P_j_k[i][3]) + np.dot(inning_score_sim(n-1, r-4, i), P_j_k[i][4])


# nを∞に発散
n = 4
U_n = []   
for r in range(R_max+1):
    U_n.append(inning_score_sim(n, r, 0))
    
#TODO 第25列が1を超えてしまっている

df_U = pd.DataFrame(U_n, columns=list(range(1,26)))
df_U

,1,2,3,4,5,6,7,8,9,10,...,16,17,18,19,20,21,22,23,24,25
0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.023272,0.000000,0.000000,0.000000,0.000000,0.261247,0.00173,0.055902,0.0,1.230847
1,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002082,0.000000,0.000000,...,0.000000,0.000000,0.061876,0.020434,0.005655,0.000000,0.00000,0.000000,0.0,0.051357
2,0.000000,0.00000,0.000000,0.000000,0.003493,0.000037,0.001193,0.000000,0.000000,0.014761,...,0.000000,0.036755,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000
3,0.000000,0.00132,0.000436,0.000121,0.000000,0.000000,0.000000,0.000000,0.008768,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000
4,0.000784,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000
5,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000
6,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000
7,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000
8,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000
9,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000


### 9イニングにおける得点分布

各状態の確率を表す行列$U_n$を用いて，１イニングにおける得点分布は導出ができた．<br>
では，$U_n$を用いてさらに9イニングにおける得点分布も考えていく．

まず，$U_n$の第25列(イニング$n$人目の打者が終了したときの，イニング終了状態における得点の確率分布)を表す($R_{max} + 1$)次元列ベクトル$u_{25}^n$を考える．

In [11]:
# u_{25}^{n}
u_25_n = df_U.iloc[:,24]
u_25_n

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
163    0.0
164    0.0
165    0.0
166    0.0
167    0.0
Name: 25, Length: 168, dtype: float64

打者$n$人目でそのイニングが終了する確率は，$u_{25}^{n}$の要素和なので，$\|u_{25}^{n}\|$と表せる．

## $L_1$ノルム

n次元ベクトル${x} = (x_1, x_2, \cdots ,x_n)$に対して,<br>
$\|{x}\|=|x_1| + |x_2| + \cdots + |x_n|$としたとき、$\|{x}\|_1を{x}$の$L_1$ノルムという。

In [12]:
u_25_n_norm = np.linalg.norm(u_25_n, ord=1)
print(u_25_n_norm)

0.0


イニング$n$人目の打者でそのイニングが終了する確率は，

$\|u_{25}^n\|_1$-$\|u_{25}^{n-1}\|_1\$

で表す事ができる．


In [13]:
# イニング4人目の打者でそのイニングが終了する確率
u_25_4_norm = 0.5718581481250001
u_25_3_norm = 0.28101137500000006

print(u_25_4_norm - u_25_3_norm)

0.29084677312500007
